In [18]:
import datetime
from functools import reduce
from IPython.display import display, clear_output
from ipywidgets import VBox, HBox
import math
import pandas as pd

import bql

from equity_scoring import EquityScoringApp
from utils_factors import BQLFunction, DescriptiveFactor, RankedFactor, ZScoreFactor, AllFactors
from utils_general import get_fundamental_data, format_floats, current, prev, fy1, fy2, ltm, ntm, sntm


bq = bql.Service()
d = bq.data
f = bq.func
all_factors = AllFactors()

### Descriptive data
---

HOW TO DEFINE/SHOW (OR NOT) YOUR DESCRIPTIVE DATA
1. Define your descriptive data (see example below);
2. Generate an instance of DescriptiveFactor (see below example):
 1. Define the name;
 2. Make sure that use_in_total_score=False to avoid it being used in the total factor scoring;
3. To add a descriptive fields use the method ".add_factor" (as in below example);
4. Add your descriptive fields using ".append" (as below) in "all_factors".

You can hide the lot from the output by simply commenting out the line where you append descriptive_fields.

In [19]:
# Descriptive Data

name = bq.data.name()
sector = bq.data.icb_sector_name()
country = bq.data.country_full_name()
market_cap = bq.data.cur_mkt_cap()

descriptive_fields = DescriptiveFactor(name='Descriptive', use_in_total_score=False)
descriptive_fields.add_factor(name='Name', bql_function=name)
descriptive_fields.add_factor(name='Sector', bql_function=sector)
descriptive_fields.add_factor(name='Country', bql_function=country)
descriptive_fields.add_factor(name='Mkt Cap (M)', bql_function=market_cap/1000000)

all_factors.append(descriptive_fields)

### Customising your factors
---
HOW TO CUSTOMISE/SHOW FACTORS OUTPUTS

1. Define your factors
 1. Create your factors (See below example) using Object Model BQL queries (as opposed to String Model);
 2. A set of overrides are already available. Should you wish to create new ones:
    - Go to "utils general" module;
    - Create your override;
    - Add it to the line "import from utils_general import ..." in the first cell.


2. Two types of scoring your factors: Zscore and Rank
   1. Z scoring and Rank scoring:
      - Create an instance of ZScoreFactor/RankedFactor class respectively (as in below example)
      - Add your scored factors using ".add_factor" 
      - and specify name, 
      - bql_function: (defined in 1.)
      - sign: (1 if large is better, -1 for the opposite)
      - relative weight: for equal weight use "1."

NOTE: the instances of ZScoreFactor/RankedFactor have the same behaviour here
(they inherit from the same class "Factor" located in "utils_factors" module)

3. Add your scored factors using ".append" (as below) in "all_factors"

In [20]:
# Value factors
# Make sure we use the fundamental ticker for the fundamental data

dividend_yield__ntm = get_fundamental_data(bq, bq.data.is_div_per_shr(**fy1)) / bq.data.px_last() * 100
ev_ebitda__ntm = get_fundamental_data(bq, bq.data.ev_to_ebitda(**ntm))

value_zscore = ZScoreFactor(name='Value Score', fillna_value=-2, use_in_total_score=True)
# value_rank = RankedFactor(name='Value Score', use_in_total_score=True)
value_zscore.add_factor(name='Dividend Yield (%) NTM', bql_function=dividend_yield__ntm, sign=1, relative_weight=2.)
value_zscore.add_factor(name='EV to EBITDA NTM', bql_function=ev_ebitda__ntm, sign=-1, relative_weight=1.)

all_factors.append(value_zscore)

# value_fields = DescriptiveFactor(name='Value', bql_functions=value_zscore.bql_functions, use_in_total_score=False)
# all_factors.append(value_fields)

In [21]:
# Quality factors
# Make sure we use the fundamental ticker for the fundamental data

roe = get_fundamental_data(bq, bq.data.RETURN_COM_EQY(**prev))
roa = get_fundamental_data(bq, bq.data.RETURN_ON_ASSET(**prev))

quality_zscore = ZScoreFactor(name='Quality Score', fillna_value=-2, use_in_total_score=True)
quality_zscore.add_factor(name='ROE', bql_function=roe, sign=1, relative_weight=1.)
quality_zscore.add_factor(name='ROA', bql_function=roa, sign=1, relative_weight=1.)

all_factors.append(quality_zscore)

# quality_fields = DescriptiveFactor(name='Quality', bql_functions=quality_zscore.bql_functions, use_in_total_score=False)
# all_factors.append(quality_fields)

In [22]:
col_defs = [
    {'headerName': 'Name', 'field': 'Name', 'width': 215, 'headerStyle': {'text-align': 'center'}},
    {'headerName': 'Sector', 'field': 'Sector', 'width': 215, 'headerStyle': {'text-align': 'center'}},
    {'headerName': 'Country', 'field': 'Country', 'width': 130, 'headerStyle': {'text-align': 'center'}},
    
    {'headerName': 'Mkt Cap (M)', 'field': 'Mkt Cap (M)', 'width': 130, 'headerStyle': {'text-align': 'center'}, 'cellStyle': {'text-align': 'right'}, 'filter': 'number'},
    
    # this is built in score
    {'headerName': 'Total Score', 'field': 'Total Score', 'width': 120, 'headerStyle': {'text-align': 'center'}, 'cellStyle': {'text-align': 'right'}, 'filter': 'number'},
    
    {'headerName': 'Value Score', 'field': 'Value Score', 'width': 120, 'headerStyle': {'text-align': 'center'}, 'cellStyle': {'text-align': 'right'}, 'filter': 'number'},
    {'headerName': 'Quality Score', 'field': 'Quality Score', 'width': 120, 'headerStyle': {'text-align': 'center'}, 'cellStyle': {'text-align': 'right'}, 'filter': 'number'},
]

In [23]:
esm = EquityScoringApp(factors=all_factors, col_defs=col_defs, connection=bq, title='Equity Scoring', description="For a guide on building Equity Scoring models, please refer to the 'Guide' notebook.")
esm

EquityScoringApp(children=(AppTitle(value='<h2 style="color: lightgrey; font-weight: bold;">Equity Scoring</h2…